In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import json

In [2]:
data = json.loads(open("data/account.json").read())
location = json.loads(open("data/location_history.json").read())
areas_visited = location['Areas you may have visited in the last two years']

In [3]:
location_list = list()

In [4]:
for j in range(len(areas_visited)):
    location_list.append([areas_visited[j]['Time'], areas_visited[j]['City'], areas_visited[j]['Region'], areas_visited[j]['Postal Code']])

In [5]:
location_list

[['2021/02/14 16:16:02 UTC', 'galloway', 'Ohio', '43119'],
 ['2021/02/12 03:38:56 UTC', 'galloway', 'Ohio', '48104'],
 ['2021/01/31 15:13:16 UTC', 'galloway', 'Ohio', '48197'],
 ['2021/01/31 04:01:26 UTC', 'galloway', 'Ohio', '48212'],
 ['2020/12/24 20:38:21 UTC', 'galloway', 'Ohio', '48242'],
 ['2021/09/11 03:27:21 UTC', 'ann arbor', 'Michigan', '48103'],
 ['2021/09/10 17:30:04 UTC', 'ann arbor', 'Michigan', '48104'],
 ['2021/10/01 17:59:06 UTC', 'ann arbor', 'Michigan', '48105'],
 ['2020/09/12 20:21:55 UTC', 'ann arbor', 'Michigan', '48109'],
 ['2021/04/16 19:45:34 UTC', 'chicago', 'Illinois', '48212'],
 ['2020/11/07 01:55:30 UTC', 'brighton', 'Michigan', '48103'],
 ['2021/10/03 17:57:22 UTC', 'brighton', 'Michigan', '48104'],
 ['2021/09/05 01:17:57 UTC', 'brighton', 'Michigan', '48109'],
 ['2021/10/04 03:09:31 UTC', 'brighton', 'Michigan', '48116'],
 ['2020/05/02 01:15:32 UTC', 'columbus', 'Ohio', '43214'],
 ['2020/06/07 02:14:07 UTC', 'columbus', 'Ohio', '43219'],
 ['2020/06/13 15:

In [6]:
location_df = pd.DataFrame (location_list, columns = ['Time', 'City','Region', 'Postal Code'])
location_df

,Time,City,Region,Postal Code
0,2021/02/14 16:16:02 UTC,galloway,Ohio,43119
1,2021/02/12 03:38:56 UTC,galloway,Ohio,48104
2,2021/01/31 15:13:16 UTC,galloway,Ohio,48197
3,2021/01/31 04:01:26 UTC,galloway,Ohio,48212
4,2020/12/24 20:38:21 UTC,galloway,Ohio,48242
...,...,...,...,...
113,2020/04/20 04:02:37 UTC,dayton,Ohio,48109
114,2020/03/02 20:30:43 UTC,dayton,Ohio,48212
115,2021/08/13 19:03:31 UTC,battle creek,Michigan,48104
116,2021/05/17 03:16:52 UTC,battle creek,Michigan,48212


In [7]:
location_df['loc'] = location_df['City'] + ', ' + location_df['Region']
location_df

,Time,City,Region,Postal Code,loc
0,2021/02/14 16:16:02 UTC,galloway,Ohio,43119,"galloway, Ohio"
1,2021/02/12 03:38:56 UTC,galloway,Ohio,48104,"galloway, Ohio"
2,2021/01/31 15:13:16 UTC,galloway,Ohio,48197,"galloway, Ohio"
3,2021/01/31 04:01:26 UTC,galloway,Ohio,48212,"galloway, Ohio"
4,2020/12/24 20:38:21 UTC,galloway,Ohio,48242,"galloway, Ohio"
...,...,...,...,...,...
113,2020/04/20 04:02:37 UTC,dayton,Ohio,48109,"dayton, Ohio"
114,2020/03/02 20:30:43 UTC,dayton,Ohio,48212,"dayton, Ohio"
115,2021/08/13 19:03:31 UTC,battle creek,Michigan,48104,"battle creek, Michigan"
116,2021/05/17 03:16:52 UTC,battle creek,Michigan,48212,"battle creek, Michigan"


In [8]:
import pgeocode
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
location = geolocator.geocode('4550 galloway, Ohio')
print(location)
print((location.latitude, location.longitude))

4550, Galloway Road, Bogart, Perkins Township, Erie County, Ohio, 44870, United States
(41.39761616382542, -82.6461944598625)


In [9]:
def loc_convert_lat(x):
    
    geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
    location = geolocator.geocode(x)
    return location.latitude

def loc_convert_lang(x):
    
    geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
    location = geolocator.geocode(x)
    return location.longitude

def loc_convert_address(x):
    
    geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
    location = geolocator.geocode(x)
    return location

In [10]:
location_df['address'] = location_df['loc'].apply(loc_convert_address)
location_df['lat'] = location_df['loc'].apply(loc_convert_lat)
location_df['long'] = location_df['loc'].apply(loc_convert_lang)
location_df

,Time,City,Region,Postal Code,loc,address,lat,long
0,2021/02/14 16:16:02 UTC,galloway,Ohio,43119,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
1,2021/02/12 03:38:56 UTC,galloway,Ohio,48104,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
2,2021/01/31 15:13:16 UTC,galloway,Ohio,48197,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
3,2021/01/31 04:01:26 UTC,galloway,Ohio,48212,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
4,2020/12/24 20:38:21 UTC,galloway,Ohio,48242,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
...,...,...,...,...,...,...,...,...
113,2020/04/20 04:02:37 UTC,dayton,Ohio,48109,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607
114,2020/03/02 20:30:43 UTC,dayton,Ohio,48212,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607
115,2021/08/13 19:03:31 UTC,battle creek,Michigan,48104,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427
116,2021/05/17 03:16:52 UTC,battle creek,Michigan,48212,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427


In [11]:
location_df.dtypes

Time            object
City            object
Region          object
Postal Code     object
loc             object
address         object
lat            float64
long           float64
dtype: object

In [12]:
location_df['Time'] = pd.to_datetime(location_df['Time'])

In [13]:
location_df['new_date'] = [d.date() for d in location_df['Time']]
location_df['new_time'] = [d.time() for d in location_df['Time']]

In [29]:
location_df['new_date'] = pd.to_datetime(location_df['new_date'])

location_df['newform_date'] = location_df['new_date'].dt.strftime('%d/%m/%Y')


In [15]:
# Single columns string conversion
location_df["new_date"] = location_df["new_date"].astype(str)
#print(location_df.dtypes)


In [16]:
location_df

,Time,City,Region,Postal Code,loc,address,lat,long,new_date,new_time,newform_date
0,2021-02-14 16:16:02+00:00,galloway,Ohio,43119,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-02-14,16:16:02,14-02-2021
1,2021-02-12 03:38:56+00:00,galloway,Ohio,48104,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-02-12,03:38:56,12-02-2021
2,2021-01-31 15:13:16+00:00,galloway,Ohio,48197,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-01-31,15:13:16,31-01-2021
3,2021-01-31 04:01:26+00:00,galloway,Ohio,48212,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-01-31,04:01:26,31-01-2021
4,2020-12-24 20:38:21+00:00,galloway,Ohio,48242,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2020-12-24,20:38:21,24-12-2020
...,...,...,...,...,...,...,...,...,...,...,...
113,2020-04-20 04:02:37+00:00,dayton,Ohio,48109,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607,2020-04-20,04:02:37,20-04-2020
114,2020-03-02 20:30:43+00:00,dayton,Ohio,48212,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607,2020-03-02,20:30:43,02-03-2020
115,2021-08-13 19:03:31+00:00,battle creek,Michigan,48104,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427,2021-08-13,19:03:31,13-08-2021
116,2021-05-17 03:16:52+00:00,battle creek,Michigan,48212,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427,2021-05-17,03:16:52,17-05-2021


In [17]:
location_df[["day", "month", "year"]] = location_df["new_date"].str.split("-", expand = True)

In [18]:
#location_df

In [19]:
location_df['Time'] = pd.to_datetime(location_df['Time'])

In [20]:
# create count column and populate with 1 for grouping and summing
location_df['count'] = 1

# create hours column to utilize later in HeatMapWithTime
location_df['hour'] = location_df['Time'].dt.hour

# view latitude, longitude and count columns, groupby latitude and longitude summing up total for each xy coordinate
dataf2 = pd.DataFrame(
    location_df.groupby(['loc', 'lat', 'long'])\
    ['count'].sum().sort_values(ascending=False))

# create list of lat/long for input into folium HeatMap
lst = dataf2.groupby(
    ['lat', 'long'])\
    .sum().reset_index().values.tolist()

In [21]:
#lst

In [22]:
import folium

# function to generate base map, has default values for zoom and tiles
def generateBaseMap(loc, zoom=12, min_zoom=0, max_zoom=18, zoom_start=10, tiles='Stamen Toner', crs='ESPG2263'):
    '''
    Function that generates a Folium base map
    Input location lat/long
    Zoom level default 12
    Tiles default to Stamen Toner
    CRS default 2263 for NYC
    '''
    return folium.Map(location=loc, 
                      control_scale=True, 
                      zoom_start=zoom,
                      tiles=tiles)
  
nyc = [40.7400, -73.985880] # generic nyc lat/lon in list format
a2 = [42.2808, -83.7430] # generic ann arbor lat/lon in list format
base_map = generateBaseMap(a2) # pass lat/lon to function
base_map

In [23]:
# import HeatMap plugin
from folium.plugins import HeatMap

# add data to basemap which we created above with custom function
HeatMap(data=lst, radius=12).add_to(base_map);

# save base map as .html
base_map.save('bike_station_HeatMap.html')

# call map 
base_map

In [24]:
location_df.head()

,Time,City,Region,Postal Code,loc,address,lat,long,new_date,new_time,newform_date,day,month,year,count,hour
0,2021-02-14 16:16:02+00:00,galloway,Ohio,43119,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-02-14,16:16:02,14-02-2021,2021,02,14,1,16
1,2021-02-12 03:38:56+00:00,galloway,Ohio,48104,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-02-12,03:38:56,12-02-2021,2021,02,12,1,3
2,2021-01-31 15:13:16+00:00,galloway,Ohio,48197,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-01-31,15:13:16,31-01-2021,2021,01,31,1,15
3,2021-01-31 04:01:26+00:00,galloway,Ohio,48212,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-01-31,04:01:26,31-01-2021,2021,01,31,1,4
4,2020-12-24 20:38:21+00:00,galloway,Ohio,48242,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2020-12-24,20:38:21,24-12-2020,2020,12,24,1,20


In [30]:
import datetime

location_df['year'] = pd.DatetimeIndex(location_df['newform_date']).year
location_df['month'] = pd.DatetimeIndex(location_df['newform_date']).month

location_df

,Time,City,Region,Postal Code,loc,address,lat,long,new_date,new_time,newform_date,day,month,year,count,hour,months_in_full,mont_year
0,2021-02-14 16:16:02+00:00,galloway,Ohio,43119,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-02-14,16:16:02,14/02/2021,2021,2,2021,1,16,February,2021 14
1,2021-02-12 03:38:56+00:00,galloway,Ohio,48104,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-02-12,03:38:56,12/02/2021,2021,12,2021,1,3,December,2021 12
2,2021-01-31 15:13:16+00:00,galloway,Ohio,48197,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-01-31,15:13:16,31/01/2021,2021,1,2021,1,15,January,2021 31
3,2021-01-31 04:01:26+00:00,galloway,Ohio,48212,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-01-31,04:01:26,31/01/2021,2021,1,2021,1,4,January,2021 31
4,2020-12-24 20:38:21+00:00,galloway,Ohio,48242,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2020-12-24,20:38:21,24/12/2020,2020,12,2020,1,20,December,2020 24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2020-04-20 04:02:37+00:00,dayton,Ohio,48109,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607,2020-04-20,04:02:37,20/04/2020,2020,4,2020,1,4,April,2020 20
114,2020-03-02 20:30:43+00:00,dayton,Ohio,48212,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607,2020-03-02,20:30:43,02/03/2020,2020,2,2020,1,20,February,2020 02
115,2021-08-13 19:03:31+00:00,battle creek,Michigan,48104,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427,2021-08-13,19:03:31,13/08/2021,2021,8,2021,1,19,August,2021 13
116,2021-05-17 03:16:52+00:00,battle creek,Michigan,48212,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427,2021-05-17,03:16:52,17/05/2021,2021,5,2021,1,3,May,2021 17


In [41]:
location_df2.dtypes

Time              datetime64[ns, UTC]
City                           object
Region                         object
Postal Code                    object
loc                            object
address                        object
lat                           float64
long                          float64
new_date               datetime64[ns]
new_time                       object
newform_date                   object
day                            object
month                           int64
year                            int64
count                           int64
hour                            int64
months_in_full                 object
mont_year                      object
conta                           int64
dtype: object

In [44]:
lista_tempo = []
for x in location_df['month']:
    
    monthinteger = x
    
    lista_tempo.append(datetime.date(1900, monthinteger, 1).strftime('%B'))

#lista_tempo

In [32]:
location_df['months_in_full'] = lista_tempo
location_df['mont_year'] = [d.split('/')[2] + " " +  d.split('/')[0] for d in location_df.newform_date]

In [38]:
weight_list = []
location_df2 = location_df
location_df2['conta'] = 1
#location_df2.head()

In [45]:
location_df2

,Time,City,Region,Postal Code,loc,address,lat,long,new_date,new_time,newform_date,day,month,year,count,hour,months_in_full,mont_year,conta
0,2021-02-14 16:16:02+00:00,galloway,Ohio,43119,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-02-14,16:16:02,14/02/2021,2021,2,2021,1,16,February,2021 14,1
1,2021-02-12 03:38:56+00:00,galloway,Ohio,48104,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-02-12,03:38:56,12/02/2021,2021,12,2021,1,3,December,2021 12,1
2,2021-01-31 15:13:16+00:00,galloway,Ohio,48197,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-01-31,15:13:16,31/01/2021,2021,1,2021,1,15,January,2021 31,1
3,2021-01-31 04:01:26+00:00,galloway,Ohio,48212,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2021-01-31,04:01:26,31/01/2021,2021,1,2021,1,4,January,2021 31,1
4,2020-12-24 20:38:21+00:00,galloway,Ohio,48242,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334,2020-12-24,20:38:21,24/12/2020,2020,12,2020,1,20,December,2020 24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2020-04-20 04:02:37+00:00,dayton,Ohio,48109,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607,2020-04-20,04:02:37,20/04/2020,2020,4,2020,1,4,April,2020 20,1
114,2020-03-02 20:30:43+00:00,dayton,Ohio,48212,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607,2020-03-02,20:30:43,02/03/2020,2020,2,2020,1,20,February,2020 02,1
115,2021-08-13 19:03:31+00:00,battle creek,Michigan,48104,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427,2021-08-13,19:03:31,13/08/2021,2021,8,2021,1,19,August,2021 13,1
116,2021-05-17 03:16:52+00:00,battle creek,Michigan,48212,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427,2021-05-17,03:16:52,17/05/2021,2021,5,2021,1,3,May,2021 17,1


In [46]:
for x in location_df2['City'].sort_values().unique:
    weight_list.append(location_df2.loc[location_df2['City'] == x,
                                      ['lat', 'long', 'City']].groupby(['lat', 'long']).sum().reset_index().values.tolist())

TypeError: 'method' object is not iterable